In [1]:
import json
with open('/Users/marcolaureano2/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [3]:
pip install yelpapi

Note: you may need to restart the kernel to use updated packages.


In [4]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [5]:
# define search
LOCATION = 'LA, CA'
TERM = 'Taco'

In [6]:
# create results-in-progress JSON file
JSON_LA_TACOS = 'Data/results_in_progress_LA_tacos.json'
JSON_LA_TACOS

'Data/results_in_progress_LA_tacos.json'

In [14]:
# Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_LA_TACOS)
## If it does not exist: 
if file_exists == False:
    
    folder = os.path.dirname(JSON_LA_TACOS)
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
    print(f'[i] {JSON_LA_TACOS} not found. Saving empty list to file.')
    
    # save an empty list
    with open(JSON_LA_TACOS,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_LA_TACOS} already exists.")

[i] Data/results_in_progress_LA_tacos.json not found. Saving empty list to file.


In [15]:
# Load previous results and use len of results for offset
with open(JSON_LA_TACOS,'r') as f:
    previous_results = json.load(f)
    
# set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')


- 0 previous results found.


In [16]:
# use yelp_api variable's search_query method to perform API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM, offset=n_results)
results.keys()


dict_keys(['businesses', 'total', 'region'])

In [17]:
# How many results
total_results = results['total']
total_results

11700

In [18]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [19]:
# import additional packages for controlling loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

585